## Ejercicio 8: Langchain
En este ejercicio, vemos un par de ejemplos con LangChain. Primero, refinaremos un prompt, y después lo usaremos para crear un resumen

In [25]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

#print(type(OPENAI_API_KEY),OPENAI_API_KEY)
#print(type(API_VERSION),API_VERSION)
#print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [26]:
import os

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Importamos langchain
Si no está instalado, descomenta la próxima línea

In [27]:
!pip -q install langchain
!pip -q install langchain-openai

##### Primero, definimos dos LLMs, uno para refinar un prompt (tarea creativa) y el segundo para utilizar dicho prompt (tarea poco creativa)

In [28]:
from langchain import PromptTemplate, LLMChain
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_ENDPOINT


#Modelo para mejorar el prompt
llm_refine = AzureChatOpenAI(azure_deployment="gpt-4o",
    api_key = OPENAI_API_KEY,
    api_version=API_VERSION,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,)

#Modelo para resumir informacion
llm_summary = AzureChatOpenAI(azure_deployment="gpt-4o",
    api_key = OPENAI_API_KEY,
    api_version=API_VERSION,
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,)


#Menos temperatura en el resumidor de informacion porque quiero que sea lo mas pegado a la realidad

##### Definimos un prompt base para generar resúmenes, y su versión para ser refinado

In [29]:
#Resumir informacion
base_summary_prompt = """
Eres un asistente experto en síntesis de información.
Tu tarea es leer el siguiente texto y generar un resumen que destaque las ideas principales.
Asegúrate de:
- Identificar los puntos clave.
- Omitir detalles secundarios.
- Mantener la coherencia y cohesión en el resumen.

Texto:
{text}

Resumen:"""


#Refinar prompts
refinement_template = """
Eres un experto en ingeniería de prompts.
Reformula, mejora y amplía con mucho más detalle el siguiente prompt de resumen para que sea más claro, específico y efectivo. El texto a resumir será del dominio de las finanzas y banca:
Prompt original:
{base_prompt}

Devuelve UNICAMENTE el prompt refinado, sin más texto antes o después.

Prompt refinado:
"""

##### Ahora usamos el módulo PromptTemplate de Langchain, y LLMChain para usar programáticamente dicho template con el LLM que hemos definido antes

Hasta que no ejecutamos el comando "run", no está llamando al LLM, solo definiendo las instrucciones (como en Spark)

In [30]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")


#PromptTemplate: Sirve para definir plantillas, esto reemplaza lo de
refine_prompt_template = PromptTemplate(
    template=refinement_template,
    input_variables=["base_prompt"]
)

# LLMChain: Permite incrustar el modelo LLM que queremos, y pegarle el prompt
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)

refined_prompt = refine_chain.run(base_prompt=base_summary_prompt)
print("Prompt refinado:\n")
display(Markdown(refined_prompt))


Prompt refinado:



Eres un asistente especializado en la síntesis avanzada de información, con un enfoque particular en el dominio de las finanzas y la banca. Tu tarea consiste en analizar el texto proporcionado y elaborar un resumen claro, preciso y bien estructurado que capture las ideas centrales del contenido. Asegúrate de seguir estas instrucciones:

1. **Identificación de puntos clave**: Extrae las ideas principales, conceptos relevantes, datos importantes, y conclusiones críticas presentadas en el texto. Enfócate en los aspectos más significativos relacionados con finanzas, banca, tendencias, estadísticas, estrategias o implicaciones económicas.

2. **Eliminación de detalles secundarios**: Omite información redundante, ejemplos extensos, anécdotas o detalles que no contribuyan directamente al entendimiento del núcleo del texto.

3. **Claridad y estructura**: Redacta el resumen de forma clara y en un lenguaje accesible, manteniendo la coherencia entre las ideas y el flujo lógico del contenido. Utiliza párrafos organizados y frases completas para garantizar que el resumen sea comprensible y profesional.

4. **Tono profesional y objetivo**: Adopta un estilo objetivo, neutral y profesional adecuado para audiencias interesadas en el ámbito financiero. Evita juicios subjetivos o interpretaciones personales.

5. **Longitud adecuada**: Asegúrate de que el resumen sea lo suficientemente breve para ser conciso, pero lo suficientemente extenso para no omitir información crucial. Apunta a una extensión de 3 a 5 oraciones por cada idea principal identificada.

6. **Formato claro**: Si hay múltiples puntos clave, organiza el resumen utilizando viñetas o enumeraciones para facilitar su lectura, salvo que se indique lo contrario.

Texto para analizar:
{text}

Resumen:

In [31]:
print(refine_prompt_template)

input_variables=['base_prompt'] input_types={} partial_variables={} template='\nEres un experto en ingeniería de prompts.\nReformula, mejora y amplía con mucho más detalle el siguiente prompt de resumen para que sea más claro, específico y efectivo. El texto a resumir será del dominio de las finanzas y banca:\nPrompt original:\n{base_prompt}\n\nDevuelve UNICAMENTE el prompt refinado, sin más texto antes o después.\n\nPrompt refinado:\n'


In [32]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")


# Crear el prompt template
refine_prompt_template = PromptTemplate(
   template=refinement_template,
   input_variables=["base_prompt"]
)


# Crear la cadena LLM con tu modelo
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)


# Ejecutar correctamente con invoke (nuevo estándar porque en versiones anteriores run() esta deprecado)
refined_prompt = refine_chain.invoke({"base_prompt": base_summary_prompt})


# Inspeccionar o extraer resultado (ajusta si la clave es diferente)
refined_prompt_str = refined_prompt["text"] if "text" in refined_prompt else refined_prompt["base_prompt"]


# Mostrar el resultado con estilo Markdown
print("Prompt refinado:\n")
display(Markdown(refined_prompt_str))


Prompt refinado:



Eres un asistente experto en análisis y síntesis de información financiera, con una habilidad avanzada para identificar y estructurar conceptos clave. Tu tarea es analizar el siguiente texto relacionado con el dominio de las finanzas y la banca y generar un resumen claro, preciso y bien estructurado que cumpla con los siguientes requisitos:

1. **Identificación de ideas principales**: Extrae y organiza los puntos clave, como conceptos financieros, métricas, tendencias, implicaciones económicas, estrategias bancarias, regulaciones o cualquier información central al tema tratado.
2. **Claridad y precisión**: Redacta el resumen utilizando un lenguaje claro y profesional, optimizado para un lector familiarizado con el ámbito financiero, pero que busca una visión general concisa y comprensible.
3. **Exclusión de detalles secundarios**: Omite información redundante, ejemplos extensos, explicaciones triviales o cualquier dato que no contribuya directamente a la comprensión de los elementos fundamentales del texto.
4. **Estructura lógica y cohesión**: Organiza el resumen en un formato fluido y coherente, asegurándote de que las ideas estén conectadas de manera lógica y se presenten en el orden que mejor facilite la comprensión del tema.
5. **Neutralidad y objetividad**: Mantén un tono imparcial y profesional, evitando opiniones o interpretaciones personales.

**Adicionalmente:**
- Si el texto incluye términos técnicos, defínelos brevemente si son esenciales para entender el contexto.
- Si el texto menciona cifras, tendencias o estadísticas relevantes, inclúyelas solo si son esenciales para respaldar las ideas principales.
- Si el texto aborda un problema o desafío, resume las posibles soluciones o implicaciones descritas.

Texto a analizar:  
{text}  

Resumen:  

##### Ahora, creamos la cadena de resumen usando el prompt refinado

In [33]:
print(refined_prompt)

{'base_prompt': '\nEres un asistente experto en síntesis de información.\nTu tarea es leer el siguiente texto y generar un resumen que destaque las ideas principales.\nAsegúrate de:\n- Identificar los puntos clave.\n- Omitir detalles secundarios.\n- Mantener la coherencia y cohesión en el resumen.\n\nTexto:\n{text}\n\nResumen:', 'text': 'Eres un asistente experto en análisis y síntesis de información financiera, con una habilidad avanzada para identificar y estructurar conceptos clave. Tu tarea es analizar el siguiente texto relacionado con el dominio de las finanzas y la banca y generar un resumen claro, preciso y bien estructurado que cumpla con los siguientes requisitos:\n\n1. **Identificación de ideas principales**: Extrae y organiza los puntos clave, como conceptos financieros, métricas, tendencias, implicaciones económicas, estrategias bancarias, regulaciones o cualquier información central al tema tratado.\n2. **Claridad y precisión**: Redacta el resumen utilizando un lenguaje c

In [34]:
refined_summary_prompt_template = PromptTemplate(
    template=refined_prompt_str,
    input_variables=["text"]
)
summary_chain = LLMChain(llm=llm_summary, prompt=refined_summary_prompt_template)

with open("financial_report.txt", "r", encoding="utf-8") as f:
    texto_extenso = f.read()

resumen = summary_chain.run(text=texto_extenso)
print("\nResumen generado:\n")
display(Markdown(resumen))


Resumen generado:



**Resumen del Informe Financiero del Grupo BBVA – Perspectiva 2024**

**1. Desempeño General y Estrategias Clave**  
En 2024, BBVA consolidó su posición en el sector financiero global gracias a estrategias de diversificación de ingresos, digitalización y gestión proactiva del riesgo. Estos enfoques permitieron resultados sólidos en rentabilidad, eficiencia operativa y resiliencia, destacando su capacidad de adaptación en un entorno económico dinámico.

---

**2. Contexto Macroeconómico y Sectorial**  
- **Inflación y Políticas Monetarias:** La desaceleración de la inflación en la zona euro permitió ajustes cautelosos en las políticas del BCE, manteniendo tasas de interés bajas que favorecieron el crédito, aunque con presión sobre márgenes financieros.  
- **Crecimiento Económico:** Crecimiento global heterogéneo, con estabilidad en economías desarrolladas y expansión en mercados emergentes impulsada por innovación tecnológica.  
- **Riesgos Geopolíticos:** Tensiones comerciales, conflictos regionales y la transición energética generaron volatilidad, condicionando estrategias de inversión y gestión de riesgos.

---

**3. Resultados Financieros**  
- **Ingresos y Rentabilidad:** Incremento del 5.3% en ingresos operativos, impulsado por la expansión de la cartera crediticia, digitalización y nuevos productos financieros.  
- **Gestión del Balance:** Sólida estructura con un ratio CET1 superior al 15% y un LCR mayor al 125%, garantizando solvencia y liquidez.  
- **Calidad de la Cartera:** Tasa de morosidad reducida (1.7%) y provisiones adecuadas para créditos dudosos, respaldadas por análisis de riesgo más rigurosos.

**Indicadores Clave:**  
- ROE: 10.1%, ROA: 0.9%, Ratio de Eficiencia: 56.7%, Crecimiento de clientes digitales: +20%.

---

**4. Transformación Digital y Sostenibilidad**  
- **Digitalización:** Inversiones en inteligencia artificial y big data mejoraron la personalización de servicios y optimizaron procesos.  
- **Sostenibilidad:** Enfoque en iniciativas ESG, incluyendo financiación de proyectos sostenibles, economía circular y reducción de huella de carbono.

---

**5. Riesgos y Desafíos Emergentes**  
- **Riesgo Crediticio:** Volatilidad en sectores específicos y mercados internacionales exige monitoreo continuo.  
- **Riesgo Tecnológico:** Incremento en amenazas cibernéticas, mitigado mediante inversiones en ciberseguridad avanzada.  
- **Impacto Regulatorio:** Adaptación a normativas más estrictas a nivel europeo y global.

---

**6. Perspectivas para 2025**  
BBVA proyecta un crecimiento moderado de ingresos operativos (4%-6%), impulsado por la expansión crediticia, digitalización y diversificación de mercados. Las prioridades incluyen:  
- Innovación tecnológica (IA y machine learning).  
- Optimización operativa mediante automatización.  
- Intensificación de políticas ESG, con énfasis en proyectos verdes y economía circular.  
- Fortalecimiento del capital y gestión de riesgos para garantizar resiliencia ante escenarios adversos.

---

**7. Conclusión**  
BBVA demostró en 2024 una sólida capacidad de adaptación, apoyada en la digitalización, diversificación de ingresos y gestión prudente del riesgo. Aunque enfrenta desafíos como la volatilidad de mercados y riesgos cibernéticos, su enfoque estratégico y compromiso con la sostenibilidad posicionan al banco como un actor clave en el sector financiero global, con perspectivas alentadoras para 2025.

In [35]:
from IPython.display import Markdown
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain.*")


# Crear el prompt template
refine_prompt_template = PromptTemplate(
   template=refinement_template,
   input_variables=["base_prompt"]
)


# Crear la cadena LLM con tu modelo
refine_chain = LLMChain(llm=llm_refine, prompt=refine_prompt_template)


# Ejecutar correctamente con invoke (nuevo estándar porque en versiones anteriores run() esta deprecado)
refined_prompt = refine_chain.invoke({"base_prompt": base_summary_prompt})


# Inspeccionar o extraer resultado (ajusta si la clave es diferente)
refined_prompt_str = refined_prompt["text"] if "text" in refined_prompt else refined_prompt["base_prompt"]


# Mostrar el resultado con estilo Markdown
print("Prompt refinado:\n")
display(Markdown(refined_prompt_str))


Prompt refinado:



Eres un asistente especializado en análisis y síntesis de información compleja, con enfoque en temas de finanzas y banca. Tu tarea consiste en leer detenidamente el texto proporcionado y elaborar un resumen claro, preciso y bien estructurado que capture las ideas principales y los conceptos clave del contenido. Asegúrate de seguir estas instrucciones de manera rigurosa:

1. **Identificación de puntos clave**: Extrae y resalta las ideas centrales, términos técnicos relevantes, tendencias, estadísticas importantes, y cualquier conclusión crítica mencionada en el texto.
2. **Omisión de detalles secundarios**: Excluye cualquier información redundante, ejemplos extensos, o datos que no sean esenciales para comprender el núcleo del mensaje.
3. **Coherencia y cohesión**: Organiza el resumen de manera lógica y fluida, conectando las ideas principales con transiciones claras para garantizar que el contenido sea fácil de entender.
4. **Contexto financiero específico**: Si el texto incluye términos especializados, conceptos técnicos o referencias específicas al sector financiero y bancario (por ejemplo: productos financieros, regulaciones, análisis de mercado, indicadores económicos, etc.), preserva su significado y precisión, explicando brevemente si es necesario para asegurar claridad.
5. **Extensión adecuada**: Mantén el resumen breve pero suficientemente detallado para transmitir el mensaje principal sin omitir información crítica.

Texto para analizar:
{text}

Resumen:

- ¿Qué ventajas le ves a usar LangChain en vez de hacer todo a mano?
- ¿Te gusta más este resumen o el que hemos hecho antes? ¿Y si cambiamos la temperatura o el modelo?

In [39]:
print(refined_prompt["text"])

Eres un asistente especializado en análisis y síntesis de información compleja, con enfoque en temas de finanzas y banca. Tu tarea consiste en leer detenidamente el texto proporcionado y elaborar un resumen claro, preciso y bien estructurado que capture las ideas principales y los conceptos clave del contenido. Asegúrate de seguir estas instrucciones de manera rigurosa:

1. **Identificación de puntos clave**: Extrae y resalta las ideas centrales, términos técnicos relevantes, tendencias, estadísticas importantes, y cualquier conclusión crítica mencionada en el texto.
2. **Omisión de detalles secundarios**: Excluye cualquier información redundante, ejemplos extensos, o datos que no sean esenciales para comprender el núcleo del mensaje.
3. **Coherencia y cohesión**: Organiza el resumen de manera lógica y fluida, conectando las ideas principales con transiciones claras para garantizar que el contenido sea fácil de entender.
4. **Contexto financiero específico**: Si el texto incluye términ

In [37]:
refined_summary_prompt_template = PromptTemplate(
    template=refined_prompt["text"],
    input_variables=["text"]
)
summary_chain = LLMChain(llm=llm_summary, prompt=refined_summary_prompt_template)

with open("financial_report.txt", "r", encoding="utf-8") as f:
    texto_extenso = f.read()

resumen = summary_chain.run(text=texto_extenso)


print("\nResumen generado:\n")
display(Markdown(resumen))


Resumen generado:



**Resumen del Informe Financiero del Grupo BBVA – Perspectiva 2024**

El Grupo BBVA ha consolidado su posición en el sector financiero global durante 2024, destacándose por su capacidad de adaptación en un entorno económico dinámico. Las estrategias de diversificación de ingresos, digitalización y gestión proactiva del riesgo han sido clave para alcanzar resultados sólidos en rentabilidad, eficiencia operativa y resiliencia.

### **1. Contexto Macroeconómico y Sectorial**
- **Inflación y Políticas Monetarias:** La desaceleración de la inflación en la zona euro permitió ajustes cautelosos en las políticas del BCE, manteniendo tasas de interés bajas que favorecieron el crédito pero presionaron los márgenes financieros.
- **Crecimiento Económico:** Las economías desarrolladas mostraron estabilidad, mientras que los mercados emergentes crecieron rápidamente gracias a la innovación tecnológica.
- **Riesgos Geopolíticos:** Las tensiones comerciales y la transición energética generaron volatilidad, condicionando estrategias de inversión.

### **2. Resultados Financieros**
- **Ingresos y Rentabilidad:** Los ingresos operativos crecieron un 5.3% interanual, impulsados por la expansión de la cartera crediticia, la digitalización y la innovación en productos financieros.
- **Estructura del Balance:** BBVA mantuvo una sólida estructura financiera, con un ratio CET1 superior al 15% y un LCR por encima del 125%, garantizando solvencia y liquidez.
- **Calidad de la Cartera:** La tasa de morosidad se redujo al 1.7%, respaldada por políticas de riesgo rigurosas y provisiones adecuadas.

### **3. Indicadores Financieros Clave**
- **ROE:** 10.1%, reflejando una sólida rentabilidad del capital.
- **ROA:** 0.9%, destacando la eficiencia en el uso de activos.
- **Ratio de Eficiencia:** 56.7%, mejorando la optimización de costes.
- **Crecimiento Digital:** Incremento del 20% en clientes digitales, reduciendo costes y mejorando la eficiencia operativa.

### **4. Transformación Digital y Sostenibilidad**
- **Innovación Tecnológica:** La integración de inteligencia artificial y big data mejoró la personalización de servicios y la eficiencia operativa.
- **Sostenibilidad:** BBVA intensificó sus iniciativas ESG, financiando proyectos sostenibles, promoviendo la economía circular y reduciendo su huella de carbono.

### **5. Riesgos y Desafíos Emergentes**
- **Riesgo Crediticio:** La volatilidad en ciertos sectores requiere un monitoreo constante de la calidad de la cartera.
- **Riesgo Tecnológico:** La digitalización aumenta la exposición a ciberamenazas, mitigadas mediante inversiones en ciberseguridad.
- **Impacto Regulatorio:** La adaptación a normativas más estrictas sigue siendo un desafío clave.

### **6. Perspectivas para 2025**
BBVA proyecta un crecimiento moderado de ingresos (4%-6%), impulsado por la expansión crediticia, la digitalización y la diversificación de mercados. La innovación tecnológica, la sostenibilidad y la optimización operativa serán pilares estratégicos, junto con el fortalecimiento del capital y la gestión del riesgo.

### **Conclusión**
En 2024, BBVA demostró resiliencia y capacidad de adaptación, consolidándose como un actor clave en el sector financiero. Su enfoque en transformación digital, sostenibilidad y gestión prudente del riesgo posiciona al banco para un crecimiento sostenido en 2025, aunque deberá enfrentar desafíos como la volatilidad de los mercados y el cumplimiento normativo.

In [38]:
print(texto_extenso)

Informe Financiero del Grupo BBVA – Perspectiva 2024

1. Resumen Ejecutivo
--------------------
Durante el ejercicio 2024, el Grupo BBVA ha consolidado su posición en el sector financiero global, 
demostrando una notable capacidad de adaptación ante un entorno económico en constante transformación. 
Las estrategias de diversificación de ingresos, la digitalización de procesos y la gestión proactiva del riesgo 
han permitido al banco obtener resultados sólidos en términos de rentabilidad, eficiencia operativa y resiliencia.
El presente informe ofrece un análisis profundo y técnico del desempeño del banco, abarcando aspectos 
macroeconómicos, análisis detallado de estados financieros, indicadores clave, estrategias de innovación, 
riesgos emergentes y proyecciones futuras.

2. Contexto Macroeconómico y Sectorial
--------------------------------------
El año 2024 se ha caracterizado por una recuperación gradual de las economías globales, aunque con 
incertidumbres derivadas de factores ge